In [1]:
#mengimport modul yang dibutuhkan

#import findspark
#findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#membuat SparkSession
appName = "Regresi di Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
#membuat skema file
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])
#membaca data dari file ke DataFrame
flightDataFrame = spark.read.csv('/root/materi/mtk-stats/Regresi Linier/dataset/flights2.csv', 
                                 schema=flightSchema, header=True)
flightDataFrame.show(25)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [3]:
#memilih kolom data yang terkait
data = flightDataFrame.select("DayofMonth", "DayOfWeek", 
                              "OriginAirportID", "DestAirportID", 
                              "DepDelay", "ArrDelay")
data.show(3)

+----------+---------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+---------------+-------------+--------+--------+
|        19|        5|          11433|        13303|      -3|       1|
|        19|        5|          14869|        12478|       0|      -8|
|        19|        5|          14057|        14869|      -4|     -15|
+----------+---------+---------------+-------------+--------+--------+
only showing top 3 rows



In [4]:
#membagi data secara random, 70% untuk training, 30% untuk testing
dataTerpisahkan = data.randomSplit([0.7, 0.3]) 
trainingData = dataTerpisahkan[0] #data training di index 0
testingData = dataTerpisahkan[1] #data testing di index 1
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 367  Testing Rows: 133


In [5]:
#mendefinisikan assembler
assembler = VectorAssembler(inputCols = [
    "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", 
    "DepDelay"], outputCol="features")
#mengubah data kita menjadi feature satu kolom, 
#menggunakan assembler yang kita definisikan
trainingDataFinal = assembler.transform(trainingData).select(
    col("features"), (col("ArrDelay").cast("Int").alias("label")))
trainingDataFinal.show(truncate=False , n=3)

+-------------------------------+-----+
|features                       |label|
+-------------------------------+-----+
|[18.0,4.0,10397.0,10721.0,5.0] |17   |
|[18.0,4.0,10397.0,10721.0,11.0]|-3   |
|[18.0,4.0,10397.0,10821.0,2.0] |-10  |
+-------------------------------+-----+
only showing top 3 rows



In [6]:
#memanggil API regresi linear dari Spark untuk algoritma kita
algoritma = LinearRegression(
    labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
#mentraining model kita dengan training data final
model = algoritma.fit(trainingDataFinal)
print ("Model regresi selesai ditraining!")

Model regresi selesai ditraining!


In [7]:
#mengubah data ke satu kolom fitur menggunakan assembler 
#yg telah kita definisikan sblmnya
testingDataFinal = assembler.transform(
    testingData).select(
    col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testingDataFinal.show(truncate=False, n=2)

+-------------------------------+---------+
|features                       |trueLabel|
+-------------------------------+---------+
|[18.0,4.0,10397.0,11042.0,11.0]|7        |
|[18.0,4.0,10397.0,11066.0,-4.0]|-4       |
+-------------------------------+---------+
only showing top 2 rows



In [8]:
#memprediksi data testing final dengan model yg telah kita training
predisksiMentah = model.transform(testingDataFinal)
#memilih kolom tertentu, yang terkait saja
prediksiFinal = predisksiMentah.select("features", "prediction", "trueLabel")
prediksiFinal.show(3)#menampilkan 3 hasil prediksi

+--------------------+------------------+---------+
|            features|        prediction|trueLabel|
+--------------------+------------------+---------+
|[18.0,4.0,10397.0...| 9.419384256965657|        7|
|[18.0,4.0,10397.0...|-5.756614999840128|       -4|
|[18.0,4.0,10397.0...|-4.761650331350605|       -1|
+--------------------+------------------+---------+
only showing top 3 rows



In [9]:
#mengimpor modul untuk mengevaluasi akurasi regresi kita
from pyspark.ml.evaluation import RegressionEvaluator
#mendefinisikan evaluator dengan menginputkan kolom dari "label" 
#dan "prediksi" data kita, serta matrik evaluasi yg kita inginkan (rmse)
evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
#menghitung rmse dg evalutor yang telah kita definisikan
rmse = evaluator.evaluate(prediksiFinal)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 16.196880382057685


In [10]:
print(testingData.take(1))

[Row(DayofMonth=18, DayOfWeek=4, OriginAirportID=10397, DestAirportID=11042, DepDelay=11, ArrDelay=7)]


In [11]:
#ambil 1 baris data list
baris1List = testingData.take(1)

baris1DataFrame = sc.parallelize(baris1List).toDF()

testingBaris1 = assembler.transform(
    baris1DataFrame).select(col("features"), 
                            (col("ArrDelay")).cast("Int").alias("trueLabel"))
#memprediksi data dengan model yg telah kita training
prediskiMentah = model.transform(testingBaris1)
prediksiFinal = prediskiMentah.select("features", "prediction", "trueLabel")
prediksiFinal.show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 18, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.